In [1]:
from FaceLandmarkDetection.src.quantization.helper import *

In [2]:
random_seed = 0
num_classes = 136
cuda_device = torch.device("cuda:0")
cpu_device = torch.device("cpu:0")

model_dir = "checkpoints"
model_filename = "resnet18_FLM.pt"
model_filepath = os.path.join(model_dir, model_filename)

quant_model_dir = "checkpoints_quantized"
quant_model_filename = "resnet18_quantized_FLM.pt"
quant_model_filepath = os.path.join(quant_model_dir, quant_model_filename)

set_random_seeds(random_seed=random_seed)

In [3]:
# Create an untrained model.
model = create_model(num_classes=num_classes)
train_dataset, val_dataset = get_data()
train_loader = make_loader(train_dataset, 64)
val_loader = make_loader(val_dataset, 32)

1111


In [4]:
model = load_model(model=model, model_filepath=model_filepath, device=cpu_device)

In [5]:
save_torchscript_model(model=model, model_dir="temp", model_filename="temp1")
ts_model = load_torchscript_model(model_filepath="temp/temp1", device=cpu_device)

MOBILE OPTIMIZE TS_MODEL

In [6]:
mobile_model_dir = "checkpoints_mobile"
mobile_model_filename = "mobile_ts_model.pt"
mobile_model_filepath = os.path.join(mobile_model_dir, mobile_model_filename)

In [7]:
import torch.utils.mobile_optimizer as mobile_optimizer
opt_model = mobile_optimizer.optimize_for_mobile(ts_model)
torch.jit.save(opt_model, mobile_model_filepath)

In [ ]:
evaluate_model(model=opt_model, test_loader=val_loader, device=cpu_device, criterion=nn.MSELoss())

In [ ]:
measure_inference_latency(model=opt_model, device=cpu_device, input_size=(32,1,224,224), num_samples=100)

MOBILE OPTIMIZE QUANTIZED MODEL

In [ ]:
mobile_model_dir = "checkpoints_mobile"
mobile_model_filename = "mobile_quantized.pt"
mobile_model_filepath = os.path.join(mobile_model_dir, mobile_model_filename)

In [ ]:
quantized_model = load_torchscript_model(model_filepath="checkpoints_quantized/resnet_static_flm.pt", device=cpu_device)
q_opt_model = mobile_optimizer.optimize_for_mobile(quantized_model)
torch.jit.save(q_opt_model, mobile_model_filepath)

In [ ]:
evaluate_model(model=q_opt_model, test_loader=val_loader, device=cpu_device, criterion=nn.MSELoss())

In [ ]:
measure_inference_latency(model=q_opt_model, device=cpu_device, input_size=(32,1,224,224), num_samples=100)